In [1]:
#IMPORTAÇÃO BIBLIOTECAS
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

In [2]:
#LEITURA DO DATASET DE ACORDÃOS DO CARF, OBTIDO ATRAVÉS DO SCRAPER
acordaos = pd.read_csv("../Dataset CARF COFINS/DADOS ACORDAOS COFINS CARF 2016-2020.csv", sep="#")

In [3]:
#CRIA FUNÇÃO PARA PADRONIZAR NOMES DOS CONTRIBUINTES PARA POSTERIOR JUNÇÃO E FILTRO DE CNPJS
def preparaNomeParaJuncao(nome):
    if isinstance(nome, float):
        print(nome)
    nomeJuncao = nome.upper().replace(
        " EM RECUPERACAO JUDICIAL", ""
    ).replace(
        " FALIDO EM RECUPERACAO JUDICIAL", ""
    ).replace(
        "S.A.", "SA"
    ).replace(
        "S/A", "SA"
    ).replace(
        "&", "E"
    )
    
    return nomeJuncao

In [4]:
print(len(acordaos))
acordaos.dropna(subset=["NOME CONTRIBUINTE"], inplace=True)
print(len(acordaos))

16087
16086


In [5]:
#OBTEM DO DATASET DE CNPJS SQL LITE APENAS AS COLUNAS E CNPJS RELACIONADOS AOS ACÓRDÃOS
acordaos["NOME CONTRIBUINTE JUNCAO"] = [preparaNomeParaJuncao(nome) for nome in acordaos["NOME CONTRIBUINTE"]]
nomesContrib = acordaos["NOME CONTRIBUINTE JUNCAO"].tolist()

con = sqlite3.connect("../Dataset CNPJ/bd_dados_qsa_cnpj.db")

listQueryFiltraCNPJs = [
    "SELECT razao_social, cnae_fiscal FROM cnpj_dados_cadastrais_pj WHERE "
        , "identificador_matriz_filial = '1' AND REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(UPPER(razao_social), "
            , "' EM RECUPERACAO JUDICIAL', '' ),' FALIDO EM RECUPERACAO JUDICIAL', ''),"
                , "'S.A.', 'SA' ), 'S/A', 'SA'),'&', 'E' ) "
                    , "IN ({1});"
]

queryFiltraCNPJs = "".join(listQueryFiltraCNPJs)
queryFiltraCNPJs = queryFiltraCNPJs.format("?", ",".join("?" * len(nomesContrib)))

cnpjs = pd.read_sql_query(queryFiltraCNPJs, con, params=nomesContrib)
con.close()
cnpjs.shape

(4249, 2)

In [6]:
#SALVA O DATASET COM OS DADOS FILTRADOS DE CNPJ
cnpjs.to_csv("../Dataset CNPJ/DADOS CNPJ.csv", index=False, sep="#")